<a href="https://colab.research.google.com/github/benjawad/AI-for-Environmental-Regulation-Analysis/blob/main/generate_legal_register_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# required dependencies

In [1]:
!pip install PyPDF2
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00


# required python libaries

In [2]:
import os
from reportlab.lib import colors
from reportlab.lib.pagesizes import landscape ,A3 , A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image ,Flowable
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch, cm , mm
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from reportlab.platypus import PageBreak


# Cover Project Page

In [3]:

def generate_legal_register_first_page (PDF_PATH = "legal_register_cover_page.pdf"):

  LOGO_PATH = "jesa_logo.png"
  PAGE_WIDTH, PAGE_HEIGHT = A4

  # --- 2. Create a Dummy Logo if it doesn't exist ---
  # This helps the script run even if the logo file is missing.
  if not os.path.exists(LOGO_PATH):
      try:
          from PIL import Image as PILImage, ImageDraw, ImageFont
          img = PILImage.new('RGB', (240, 70), color='white')
          d = ImageDraw.Draw(img)
          # Use a common bold font if available, otherwise default
          try:
              font = ImageFont.truetype("arialbd.ttf", 50)
          except IOError:
              font = ImageFont.load_default()
          d.text((10, 5), "JESA", fill=colors.HexColor("#1F497D"), font=font)
          img.save(LOGO_PATH)
          print(f"Created a dummy logo: {LOGO_PATH}")
      except Exception as e:
          print(f"Warning: Could not create a dummy logo. Please provide {LOGO_PATH}. Error: {e}")

  # --- 3. Document Setup ---
  doc = SimpleDocTemplate(
          PDF_PATH,
          pagesize=A4,
          rightMargin=0.5 * inch,
          leftMargin = 0.5* inch,
          topMargin=0.5 * inch,
          bottomMargin=0.75 * inch,
      )
  elements = []
  # Calculate the available width for content on the page
  content_width = PAGE_WIDTH - doc.leftMargin - doc.rightMargin

  # --- 4. Define Paragraph Styles ---
  # Style for the main title in the top blue bar
  header_style = ParagraphStyle(
      name="Header",
      fontName="Helvetica-Bold",
      fontSize=12,
      textColor=colors.white,
      leading=16,
      leftIndent=10
  )

  # Style for the "Purpose of this register..." text
  subheader_style = ParagraphStyle(
      name="Subheader",
      fontName="Helvetica",
      fontSize=7,
      textColor=colors.black,
      leading=12,
      spaceAfter=6,
  )

  # Style for the blue field labels (e.g., "PROJECT No:")
  label_style = ParagraphStyle(
      name="Label",
      fontName="Helvetica-Bold",
      fontSize=6,
      textColor=colors.white,
      leftIndent=4,
      leading=12,
  )

  # Style for the text inside the value boxes (e.g., "Q37440")
  value_style = ParagraphStyle(
      name="Value",
      fontName="Helvetica-bold",
      fontSize=6,
      textColor=colors.black,
      leading=12,
  )

  # Style for table text
  table_style = ParagraphStyle(
      name="TableText",
      fontName="Helvetica",
      fontSize=8,
      textColor=colors.black,
      leading=10,
      alignment=1,  # Center alignment
  )

  table_header_style = ParagraphStyle(
      name="TableHeader",
      fontName="Helvetica-Bold",
      fontSize=8,
      textColor=colors.white,
      leading=10,
      alignment=1,  # Center alignment
  )

  # --- 5. Build Combined Header and Purpose Box ---
  # This single table creates the continuous border effect.
  title_para = Paragraph(
      "Sustainable Project Delivery - Legal Register - Chemical additives plant",
      header_style
  )
  logo_img = Image(LOGO_PATH, width=80, height=25)
  purpose_para = Paragraph(
      "Purpose of this register is to record the regulatory requirements that need to be complied with by the project. "
      "The register provides traceability of the action that has been taken to address the requirement.",
      subheader_style
  )

  # The table has two rows: one for the header, one for the purpose text.
  combined_header_table = Table(
      [
          [title_para, logo_img],     # First row: Title and Logo
          [purpose_para, None]        # Second row: Purpose text (spans both columns)
      ],
      colWidths=[content_width - 88, 88],
      rowHeights=[12*mm, None] # First row has fixed height, second is auto
  )

  combined_header_table.setStyle(TableStyle([
      # Span the purpose cell across the whole width
      ('SPAN', (0, 1), (1, 1)),

      # Background Colors
      ('BACKGROUND', (0, 0), (0, 0), colors.HexColor("#1F497D")), # Blue for title
      ('BACKGROUND', (1, 0), (1, 0), colors.white),             # White for logo

      # Alignment
      ('VALIGN', (0, 0), (-1, 0), 'MIDDLE'), # Middle-align the header row
      ('ALIGN', (1, 0), (1, 0), 'CENTER'),   # Center the logo

      # Borders and Lines
      ('BOX', (0, 0), (-1, -1), 1, colors.black), # Main outer border
      ('LINEBELOW', (0, 0), (-1, 0), 1, colors.black), # Line under the header
      ('LINEBEFORE', (1, 0), (1, 0), 1, colors.black), # Vertical line next to logo

      # Padding for the purpose text cell
      ('TOPPADDING', (0, 1), (-1, 1), 5),
      ('BOTTOMPADDING', (0, 1), (-1, 1), 5),
      ('LEFTPADDING', (0, 1), (-1, 1), 5),
      ('RIGHTPADDING', (0, 1), (-1, 1), 5),
  ]))

  elements.append(combined_header_table)
  elements.append(Spacer(1, 8*mm))

  # --- 6. Build Project Detail Fields ---
  fields = [
      ("PROJECT No:", "Q37440"),
      ("PROJECT TITLE:", "Chemical additives plant"),
      ("JESA DOCUMENT No:", "Q37440-00-EN-REG-00001"),
      ("ELECTRONIC FILE LOCATION:", "N/A"),
      ("NOTES:", "N/A"),
  ]

  for label, val in fields:
      # Create the full-width blue bar for the label
      label_para = Paragraph(label, label_style)
      label_table = Table([[label_para]], colWidths=[content_width], rowHeights=[7*mm])
      label_table.setStyle(TableStyle([
          ('BACKGROUND', (0, 0), (-1, -1), colors.HexColor("#1F497D")),
          ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
      ]))
      elements.append(label_table)
      elements.append(Spacer(1, 1.5*mm)) # Small space between label and value box

      # Create the smaller, bordered box for the value
      value_para = Paragraph(val, value_style)
      # The value box has a fixed width (50% of the page content width)
      value_box = Table([[value_para]], colWidths=[content_width / 2])
      value_box.setStyle(TableStyle([
          ('BOX', (0, 0), (-1, -1), 0.5, colors.grey),
          ('LEFTPADDING', (0, 0), (-1, -1), 4),
          ('TOPPADDING', (0, 0), (-1, -1), 2),
          ('BOTTOMPADDING', (0, 0), (-1, -1), 4),
      ]))
      elements.append(value_box)

      # Add a larger space before the next field starts
      elements.append(Spacer(1, 5*mm))

  # --- 7. Add Bottom Status Table ---
  # Add significant space to push content towards bottom of page
  elements.append(Spacer(1, 40*mm))

  # Create the Originator/Issue Date table (first row)
  originator_table_data = [
      [
          Paragraph("Originator:", table_style),
          Paragraph("Y.Hosni", table_style),
          Paragraph("Issue Date:", table_style),
          Paragraph("18-Jun-25", table_style)
      ]
  ]

  originator_table = Table(
      originator_table_data,
      colWidths=[content_width * 0.15, content_width * 0.35, content_width * 0.15, content_width * 0.35]
  )

  originator_table.setStyle(TableStyle([
      ('BOX', (0, 0), (-1, -1), 1, colors.black),
      ('INNERGRID', (0, 0), (-1, -1), 0.5, colors.black),
      ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
      ('LEFTPADDING', (0, 0), (-1, -1), 3),
      ('RIGHTPADDING', (0, 0), (-1, -1), 3),
      ('TOPPADDING', (0, 0), (-1, -1), 4),
      ('BOTTOMPADDING', (0, 0), (-1, -1), 4),
  ]))

  elements.append(originator_table)

  # Create the Document Status header
  status_header_data = [
      [Paragraph("DOCUMENT STATUS", table_header_style)]
  ]

  status_header_table = Table(
      status_header_data,
      colWidths=[content_width]
  )

  status_header_table.setStyle(TableStyle([
      ('BACKGROUND', (0, 0), (-1, -1), colors.HexColor("#1F497D")),
      ('BOX', (0, 0), (-1, -1), 1, colors.black),
      ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
      ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
      ('TOPPADDING', (0, 0), (-1, -1), 6),
      ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
  ]))

  elements.append(status_header_table)

  # Create the main status table with headers and data
  status_table_data = [
      # Data row B
      [
          Paragraph("B", table_style),
          Paragraph("18-Jun-25", table_style),
          Paragraph("Issued for Review (IFR)", table_style),
          Paragraph("Y.Hosni", table_style),
          Paragraph("S.El Alem", table_style),
          Paragraph("J.Alaoui Sosse", table_style),
          Paragraph("S. Paresh", table_style)
      ],
      # Data row A
      [
          Paragraph("A", table_style),
          Paragraph("11-Mar-25", table_style),
          Paragraph("Issued for Internal Review (IIR)", table_style),
          Paragraph("I.Issa Issaka", table_style),
          Paragraph("S.El Alem", table_style),
          Paragraph("J.Alaoui Sosse", table_style),
          Paragraph("S. Salim", table_style)
      ],
      [
          Paragraph("REV", table_style),
          Paragraph("DATE", table_style),
          Paragraph("DESCRIPTION", table_style),
          Paragraph("BY", table_style),
          Paragraph("CHKD", table_style),
          Paragraph("D.APPD", table_style),
          Paragraph("P.APPD", table_style)
      ],
  ]

  status_table = Table(
      status_table_data,
      colWidths=[
          content_width * 0.06,   # REV
          content_width * 0.12,   # DATE
          content_width * 0.35,   # DESCRIPTION
          content_width * 0.15,   # BY
          content_width * 0.12,   # CHKD
          content_width * 0.12,   # D.APPD
          content_width * 0.08    # P.APPD
      ]
  )
  num_rows = len(status_table_data)

  status_table.setStyle(TableStyle([
      ('BOX', (0, 0), (-1, -1), 1, colors.black),
      ('INNERGRID', (0, 0), (-1, -1), 0.5, colors.black),
      ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
      ('LEFTPADDING', (0, 0), (-1, -1), 3),
      ('RIGHTPADDING', (0, 0), (-1, -1), 3),
      ('TOPPADDING', (0, 0), (-1, -1), 4),
      ('BOTTOMPADDING', (0, 0), (-1, -1), 4),
      # Light blue background for header row only
      ('BACKGROUND', (0, num_rows - 1), (-1, num_rows - 1), colors.HexColor("#E6F3FF")),

  ]))

  elements.append(status_table)

  # Copyright notice
  copyright_para = Paragraph(
      "© Copyright 2021 JESA Group. No part of this document or the information it contains may be reproduced or transmitted in any form or by any means electronic or mechanical, including photocopying, recording, or by any information storage and retrieval system, without permission in writing from JESA. JESA.com",
      ParagraphStyle(
          name="Copyright",
          fontName="Helvetica-bold",
          fontSize=7,
          textColor=colors.black,
          leading=9,
          alignment=0,
          spaceAfter=0,
      )
  )

  elements.append(Spacer(1, 4*mm))
  elements.append(copyright_para)

  # --- 8. Render the PDF ---
  doc.build(elements)
  print(f"✅ PDF successfully created: {PDF_PATH}")

generate_legal_register_first_page()

✅ PDF successfully created: legal_register_cover_page.pdf


# Header

In [4]:
def header_footer(canvas, doc):
        jesa_blue = colors.Color(red=0/255, green=51/255, blue=102/255)

        canvas.saveState()
        page_w, page_h = landscape(A3)

        line_y = page_h - doc.topMargin + 2.15 * cm
        canvas.setStrokeColor(jesa_blue)
        canvas.setLineWidth(1)
        canvas.line(doc.leftMargin, line_y, page_w - doc.rightMargin, line_y)



        # --- Draw top separator line first ---
        line_y = page_h - doc.topMargin + 0.5 * cm
        canvas.setStrokeColor(jesa_blue)
        canvas.setLineWidth(1)
        canvas.line(doc.leftMargin, line_y, page_w - doc.rightMargin, line_y)

        # --- Logo, at top‐left just above the line ---
        logo_path = 'jesa_logo.png'
        logo_w = 4.0 * cm
        logo_h = 3.0 * cm
        logo_y = line_y + 0.2 * cm
        if os.path.exists(logo_path):
            logo = Image(logo_path, width=logo_w, height=logo_h)
            logo.drawOn(canvas, doc.leftMargin, logo_y)
        else:
            canvas.setFont('Helvetica-Bold', 30)
            canvas.setFillColor(jesa_blue)
            canvas.drawString(doc.leftMargin, logo_y + 0.4*cm, "JESA")

        # --- Left table: Project Name / Customer / Document Title ---
        info_y = line_y - 0.2 * cm
        left_data = [
            ['Project Name:', 'Chemical additives plant'],
            ['Customer:',     'NOVADDIX'],
            ['Document Title:', 'Sustainable Project Delivery - Legal Register - Chemical additives plant']
        ]
        left_col_w = [(doc.width - logo_w - 0.5*cm)*0.1,
                      (doc.width - logo_w - 0.5*cm)*0.8]

        left_tbl = Table(left_data, colWidths=left_col_w)
        left_tbl.setStyle(TableStyle([
            ('FONTNAME',    (0,0), (0,-1),   'Helvetica-Bold'),
            ('FONTNAME',    (1,0), (1,-1),   'Helvetica'),
            ('FONTSIZE',    (0,0), (-1,-1),  6),
            ('VALIGN',      (0,0), (-1,-1), 'TOP'),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING',(0,0), (-1,-1), 0),
        ]))
        left_x = doc.leftMargin + logo_w + 0.5 * cm
        left_tbl.wrapOn(canvas, doc.width, doc.topMargin)
        left_tbl.drawOn(canvas, left_x, info_y)

        # --- Right table: Ref code / Rev / Page ---
        right_data = [
            ['Q37440-00-EN-REG-00001'],
            ['REV A'],
            ['Page %d' % canvas.getPageNumber()]
        ]
        right_col_w = 3 * cm
        right_tbl = Table(right_data, colWidths=[right_col_w])
        right_tbl.setStyle(TableStyle([
            ('FONTNAME',    (0,0), (-1,-1),   'Helvetica-Bold'),
            ('FONTNAME',    (0,1), (0,1),     'Helvetica-Bold'),
            ('FONTSIZE',    (0,0), (-1,-1),    6),
            ('ALIGN',       (0,0), (-1,-1), 'RIGHT'),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING',(0,0), (-1,-1), 0),
        ]))
        right_x = page_w - doc.rightMargin - right_col_w
        right_tbl.wrapOn(canvas, doc.width, doc.topMargin)
        right_tbl.drawOn(canvas, right_x, info_y)

        canvas.restoreState()

# Final Page 2

In [11]:
def generate_commitment_register_landscape():
    filename = "Commitment_Register_Landscape.pdf"
    page_width, page_height = landscape(A3)

    # Setup document with balanced side margins
    doc = SimpleDocTemplate(
        filename,
        pagesize=landscape(A3),
        rightMargin=1.2 * inch,
        leftMargin = 1.2* inch,
        topMargin=2.25 * inch,
        bottomMargin=0.75 * inch,
    )

    # Use full content width for tables
    usable_width = doc.width

    story = []
    styles = getSampleStyleSheet()

    # Custom color for JESA blue
    jesa_blue = colors.Color(red=0/255, green=51/255, blue=102/255)


    # Main heading style
    styles.add(ParagraphStyle(
        name='MainHeading',
        fontName='Helvetica-Bold',
        fontSize=8,  # Maintained as specified
        leading=10,   # Tightened line spacing
        textColor=jesa_blue,
        spaceBefore=8,  # Added space above heading
        spaceAfter=4    # Space below heading
    ))

    # Sub-heading style
    styles.add(ParagraphStyle(
        name='SubHeading',
        fontName='Helvetica-Bold',
        fontSize=7,   # Maintained as specified
        leading=4,    # Tightened line spacing
        textColor=colors.black,
        spaceBefore=6,  # Space above subheading
        spaceAfter=0    # Space below subheading
    ))

    # Body text style
    body_style = styles['BodyText']
    body_style.fontName = 'Helvetica'
    body_style.fontSize = 7  # Maintained as specified
    body_style.leading = 10  # Tightened line spacing
    body_style.alignment = 4  # Justified text
    body_style.spaceAfter = 4  # Reduced space after paragraph

    # List item style
    styles.add(ParagraphStyle(
        name='ListItem',
        parent=body_style,
        leftIndent=0 * inch,
        spaceBefore=0,
        spaceAfter=0  # Space between list items
    ))
    # --- Build document content ---
    story.append(Paragraph("1. Objective", styles['MainHeading']))
    story.append(Paragraph(
        """The Legal Register is a listing of acts and regulations, including permits and approvals, applicable to the project or contract. The Legal Register is most useful on contracts and major projects where a significant number of regulations apply. The regulations can be sorted
by Project Phase, Discipline, or Activity, and thus more easily managed. The objective of using a Legal Register is to be proactive in managing regulatory requirements and therefore be within compliance for the duration of the project/contract. Normally, the Sustainable
Project Delivery (SPD) Lead will create and manage the Register, and provide updates to the Project Team on a regular basis. The SPD Lead is included in project activities such as status meetings to constantly inform relevant parties of their upcoming regulatory
obligations on behalf of the customer.
Creating and maintaining a Legal Register is agree between JESA and the Customer before project initiation and often during project development/Select phase.""",
        styles['BodyText']
    ))

    story.append(Paragraph("2. Principals", styles['MainHeading']))
    story.append(Paragraph(
        """• The Legal Register must be reviewed and approved by the Customer before use on a project or contract, as the Customer will own the Register.
• The SPD Lead, or designate, shall work with the Discipline Leads and project manager(s) to populate and maintain an accurate status of each legal requirement on the register.
• The SPD Lead, or designate, shall work with the Discipline Leads to determine which legal requirements must be incorporated into the design and who will be responsible for completing required actions.
• The Legal Register can be used to create a Legal Matrix. A Legal Matrix is a table used to visually summarize the Legal Register. By listing project activities in a column and key legislation in a row across the top, markers can be placed to indicate
all legislation that will impact a certain project activity.""",
        styles['ListItem']
    ))

    story.append(Paragraph("3. Instruction", styles['MainHeading']))
    story.append(Paragraph("<b>Initiating and Customizing the Commitment Register</b>", styles['SubHeading']))
    story.append(Paragraph("The Project Manager / Project Engineering Manager or designate, shall:", styles['BodyText']))
    story.append(Paragraph("- work with the Customer to populate the Register and classify the commitments.", styles['ListItem']))
    story.append(Paragraph("- be responsible for ensuring commitments are registered and communicated to the appropriate party (e.g. the discipline lead responsible for incorporating a given commitment within the project scope of work).", styles['ListItem']))
    story.append(Paragraph("The Commitment Register is designed to be customizable to suit the project's commitment tracking needs. Columns such as 'Affected areas or processes' should be customized to reflect the project.", styles['BodyText']))

    story.append(Paragraph("<b>Register Maintenance</b>", styles['SubHeading']))
    story.append(Paragraph("The Project Manager, Project Engineering Manager or designate, shall work with the Discipline Leads to maintain an accurate status of each commitment on the register. The register shall be updated as needed and controlled properly so only the most recent version is available to the project team. Sufficient hours shall be included in the project budget for register maintenance.", styles['BodyText']))
    story.append(Paragraph("<b>Technical Review</b>", styles['SubHeading']))
    story.append(Paragraph("The Commitment Register shall be reviewed by the Project Management Team and approved by the Customer at an agreed frequency for the project. After each review and approval the signed Commitment Register shall be converted to PDF and saved while updates continue in the live register.", styles['BodyText']))
    story.append(Paragraph("<b>Other Considerations</b>", styles['SubHeading']))
    story.append(Paragraph("The commitments and other registers (Legal, Sustainable Solutions Database), are normally created in conjunction with the Sustainability Steering Committee (SSC). The SSC is comprised of sustainability stakeholders from the customer (e.g. public relations, environmental advisors, regulatory contacts, operations manager) and JESA (e.g. Sustainability Lead, environmental scientists).", styles['BodyText']))

    story.append(Paragraph("4. References", styles['MainHeading']))
    for ref in [
        "Safe and Sustainable Engineering for Asset Lifecycle (SEAL) Standard (MS-E9-STD-00017)",
        "Sustainable Project Delivery - Legal Register (MS-E9-TEM-00053)",
        "Sustainable Solutions Standard (MS-FM-STD-00158)",
    ]:
        story.append(Paragraph(ref, styles['ListItem']))
    story.append(Spacer(1, 0.2 * inch))

    story.append(Paragraph("5. Abbreviations", styles['MainHeading']))
    final_table_data = [
        ['ABH', 'Agence du Bassin Hydraulique', 'EHS', 'Environment, Health & Safety'],
        ['BAT', 'Best Available Technologies', 'HR', 'Human Resources'],
        ['CRI', 'Centre Régional d\'Investissement', 'IASE', 'Health Safety & Environment'],
        ['ONG', 'Organisation Non Gouvernementale', 'PSE', 'Programme de Suivi et de Surveillance Environnemental'],
        ['OCP', 'Office Chérifien des Phosphates', 'SDG', 'Sustainable Development Goals'],
        ['', '', 'SEAL', 'Safe and Sustainable Engineering for Asset Lifecycle'],
    ]
    # Spread to full width: e.g. 10%,35%,10%,45%
    abbrev_col_widths = [usable_width * 0.10, usable_width * 0.35, usable_width * 0.10, usable_width * 0.45]
    abbreviations_table = Table(final_table_data, colWidths=abbrev_col_widths)
    abbreviations_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 6),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('FONTNAME', (2, 0), (2, -1), 'Helvetica-Bold'),
        ('LEFTPADDING', (0,0), (-1,-1), 5),
        ('RIGHTPADDING', (0,0), (-1,-1), 5),
        ('TOPPADDING', (0,0), (-1,-1), 3),
        ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ]))
    story.append(abbreviations_table)
    return story

# Page 3


In [39]:


# --- Class for Vertical Text (No Change) ---
class VerticalText(Flowable):
    """A custom flowable to draw text rotated by 90 degrees."""
    def __init__(self, text, font_name='Helvetica-Bold', font_size=4):
        Flowable.__init__(self)
        self.text = text
        self.font_name = font_name
        self.font_size = font_size

    def draw(self):
        canvas = self.canv
        canvas.saveState()
        canvas.setFont(self.font_name, self.font_size)
        canvas.rotate(90)
        canvas.drawString(5, -self.font_size, self.text)
        canvas.restoreState()

    def wrap(self, available_width, available_height):
        text_width = stringWidth(self.text, self.font_name, self.font_size)
        return (self.font_size, text_width + 10)

def generate_commitment_register_pdf():
    """
    Generates a PDF file with a detailed table matching the provided image,
    including all data rows and bulleted lists.
    """
    output_filename = "commitment_register_full.pdf"
    pagesize = landscape(A3)
    margin = 1.65* inch
    doc_width, doc_height = pagesize
    table_width = doc_width - 2 * margin


    doc = SimpleDocTemplate(
        output_filename,
        pagesize=landscape(A3),
        rightMargin=1.2 * inch,
        leftMargin = 1.2* inch,
        topMargin=2.25 * inch,
        bottomMargin=0.75 * inch,
    )


    # --- Paragraph Styles for Cell Content ---
    styles = getSampleStyleSheet()
    # Style for regular text, allowing line breaks
    cell_style = ParagraphStyle(
        name='CellStyle',
        parent=styles['Normal'],
        alignment=TA_LEFT,
        fontSize=8,
        leading=10,
    )
    # Style for centered text
    center_cell_style = ParagraphStyle(
        name='CenterCellStyle',
        parent=cell_style,
        alignment=TA_CENTER,
        fontSize=4,

    )
    # This style will control the header text's appearance and enable wrapping.
    header_style = ParagraphStyle(
        name='HeaderStyle',
        parent=styles['Normal'],
        fontName='Helvetica-Bold',
        fontSize=4,
        leading=8,
        alignment=TA_CENTER,
        textColor=colors.white,
    )

    # --- ADDED: Helper function to create header paragraphs ---
    def create_header_paragraph(text , style = header_style ):
        # Replace manual newlines with <br/> tags for the Paragraph object

        return Paragraph(text.replace('\n', '<br/>'), style)

    # Helper function to create paragraphs, handling line breaks and bullets
    def create_paragraph(text, style=center_cell_style):
        # Replace newlines with <br/> and asterisks with bullet points
        text = text.replace('\n', '<br/>').replace('* ', '&bull; ')
        return Paragraph(text, style)

    # --- Table Headers (No Change) ---
    header1_str = ['Phase', 'Activity/Aspect', 'Impacts', 'Jurisdiction', 'Type', 'Legal Requirement', 'Date', 'Description', 'Task', 'Responsibility', 'Comments']
    header1 = [create_header_paragraph(h ,ParagraphStyle(  name = "top_header",parent=header_style , fontSize=8 ) ) if h else '' for h in header1_str]

    # CORRECTED: header2 must have 30 elements to match your column layout.
    header2 = [
        "General Environmental & Sustainability Regulations"
    ]
    # header2 = [create_header_paragraph(h) if h else '' for h in header2_str]

    # header3_str_and_obj = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', VerticalText('Preparation/Construction'), VerticalText('Input Management'), VerticalText('Operation'), VerticalText('Discharge Management'), VerticalText('Off-Sites'), VerticalText('Other'), VerticalText('Fungibility'), VerticalText('OPEX'), VerticalText('Health & Safety'), VerticalText('Social'), VerticalText('Economic'), VerticalText('Environmental'), VerticalText('Regulatory'), '', '']
    # header3 = [create_header_paragraph(h) if isinstance(h, str) and h else h for h in header3_str_and_obj]

    # --- All Data Rows Transcribed from the Image ---
    data_rows = [
        # Row 1: SDG1
        [
            create_paragraph("United Nations\nSustainable Development\nGoals", center_cell_style),
            create_paragraph("SDG1: End Poverty", center_cell_style),
            create_paragraph("Commitment", center_cell_style),
            create_paragraph("The project can help to reduce poverty through direct/indirect\nemployment during construction and operation phases,\nsourcing goods, supporting the development of small firms and\nservices locally and the payment of taxes and royalties, which\nenable the development of essential social and economic\ninfrastructure", center_cell_style),
            create_paragraph("Construction/\nOperation", center_cell_style),
            create_paragraph("Low", center_cell_style),
            create_paragraph("In Progress", center_cell_style),
            create_paragraph("During\nconstruction\nand operation\nphases", center_cell_style),
            create_paragraph("Board of Directors -\nClient", center_cell_style),
            create_paragraph("-", center_cell_style),
            create_paragraph("-", center_cell_style),
            create_paragraph("* Community Engagement policy\n(OCP)\n* ESIA & PSSE\n* Responsible Local\nCommunities Relations Policy\n(OCP)"),
            create_paragraph("Improve economic and social\nconditions in the project area",center_cell_style),
            create_paragraph("Ministry of Economy and Finance", center_cell_style),
            '', 'X', 'X', '', '', '', '', 'X', '', 'X', 'X', 'X', '', '', '', ''
        ],
        # Row 2: SDG2
        [
            create_paragraph("United Nations\nSustainable Development\nGoals", center_cell_style),
            create_paragraph("SDG2: End hunger, achieve\nfood security and improved\nnutrition and promote\nsustainable agriculture", center_cell_style),
            create_paragraph("Commitment", center_cell_style),
            create_paragraph("The chemical additives are fundamental for the fertilizers\nproduction which contribute to the improvement of the\nagriculture system throughout the globe and thus provides\nfarmers with essential nutrients that help them to\nimprove their yields, income & livelihood."),
            create_paragraph("Design/\nConstruction/\nOperation", center_cell_style),
            create_paragraph("Low", center_cell_style),
            create_paragraph("In Progress", center_cell_style),
            create_paragraph("During Operational\nphase", center_cell_style),
            create_paragraph("Board of Directors -\nClient", center_cell_style),
            '', '',
            create_paragraph("* Community Engagement policy\n(OCP)\n* ESIA & PSSE"),
            create_paragraph("Improve crop yields"),
            create_paragraph("* Ministry of Agriculture,\nmaritime Fishing, Rural\nDevelopment, Water and\nForests"),
            '', 'X', 'X', '', '', '', '', '', '', '', 'X', 'X', '', '', '', ''
        ],
        # Row 3: SDG3
        [
            create_paragraph("United Nations\nSustainable Development\nGoals", center_cell_style),
            create_paragraph("SDG3: Ensure healthy lives\nand promote well-being for\nall at all ages and\nsurrounding population", center_cell_style),
            create_paragraph("Commitment", center_cell_style),
            create_paragraph("The project will ensure healthy lives and promote well being for all\nemployees and the surrounding population through the application\nof the best practices in occupational health and safety, design of\ninfrastructure that will be used by the surrounding population,\nrevalorization and treatment of all air emissions, liquid effluents,\nland and wastes."),
            create_paragraph("Design/\nConstruction/\nOperation", center_cell_style),
            create_paragraph("High", center_cell_style),
            create_paragraph("In Progress", center_cell_style),
            create_paragraph("During Design/\nConstruction and\nOperational\nphases", center_cell_style),
            create_paragraph("Safety in Design\nEngineering", center_cell_style),
            create_paragraph("HSE Client", center_cell_style),
            '',
            create_paragraph("* Occupational health and\nsafety policy (OCP)\n* Responsible local communities\nrelations policy (OCP)\n* Working conditions policy\n(OCP)\n* ESIA &PSSE"),
            create_paragraph("* Prevention of health and safety\nimpacts for employees and\nthe surrounding population due\nto poor management of project\nemissions, effluents and waste"),
            create_paragraph("* Ministry of Justice\n* Ministry of Energy Transition\nand Sustainable Development -\nDepartment of Sustainable\nDevelopment\n* Ministry of Economic\nInclusion, small Business,\nEmployment and skills"),
            '', 'X', '', 'X', 'X', '', '', '', '', 'X', 'X', '', 'X', '', '', ''
        ],
        # Row 4: SDG5
        [
            create_paragraph("United Nations\nSustainable Development\nGoals", center_cell_style),
            create_paragraph("SDG5: Achieve gender\nequality and empower all\nwomen and girls", center_cell_style),
            create_paragraph("Commitment", center_cell_style),
            create_paragraph("The project must align with OCP group commitment to enable\nits employees to achieve their full potential Opportunities are\nindifference, as the context of OCP through its diversity & inclusion\npolicy, there is no discrimination on any criteria:\n* Gender\n* Disability\n* Age\n* Academic background,\n* Nationality\n* Cultural & Religion"),
            create_paragraph("Construction/\nOperation", center_cell_style),
            create_paragraph("Low", center_cell_style),
            create_paragraph("In Progress", center_cell_style),
            create_paragraph("During Construction\nand Operation\nphase", center_cell_style),
            create_paragraph("HR Client", center_cell_style),
            create_paragraph("Human Resources", center_cell_style),
            create_paragraph("Human Resources\nDirector", center_cell_style),
            create_paragraph("* Diversity and inclusion policy\n(OCP)\n* General human capital policy\n(OCP)"),
            create_paragraph("Ensure gender equality"),
            create_paragraph("* Ministry of solidarity, social\nintegration and Family\n* CNDH\n* Ministry of Justice"),
            '', '', '', 'X', '', '', '', '', '', '', 'X', '', '', '', '', ''
        ],
        # Row 5: SDG6
        [
            create_paragraph("United Nations\nSustainable Development\nGoals", center_cell_style),
            create_paragraph("SDG6: Ensure availability\nand sustainable\nmanagement of water and\nsanitation", center_cell_style),
            create_paragraph("Commitment", center_cell_style),
            create_paragraph("The project will use only non-conventional water for domestic\nconsumption to the extent of its ability\n* Sea water desalination\n* rain water\nSanitary & domestic wastewater will be treated before being\ndischarged into the natural environment."),
            create_paragraph("Design /\nConstruction /\nOperation", center_cell_style),
            create_paragraph("High", center_cell_style),
            create_paragraph("In Progress", center_cell_style),
            create_paragraph("During Design/\nConstruction and\nOperational phases", center_cell_style),
            create_paragraph("Environmental\nProcess Engineering", center_cell_style),
            create_paragraph("Civil Engineering\nProcess Engineering", center_cell_style),
            create_paragraph("Operation Client", center_cell_style),
            create_paragraph("* Water management policy\n(OCP)\n* Drainage sewer and water\nmanagement plan"),
            create_paragraph("Water consumption\nWastewater sanitation"),
            create_paragraph("* Ministry of Energy Transition and\nSustainable Development -\nDepartment of Sustainable\nDevelopment\n* Ministry of Public Works and\nWater - Water Resources and\nWarning Department\n* ONEE"),
            '', '', 'X', 'X', 'X', '', '', '', '', 'X', 'X', '', 'X', '', '', ''
        ],
        # Row 6: SDG7
        [
            create_paragraph("United Nations\nSustainable Development\nGoals", center_cell_style),
            create_paragraph("SDG7: Ensure access to\naffordable, reliable,\nsustainable and modern\nenergy", center_cell_style),
            create_paragraph("Commitment", center_cell_style),
            create_paragraph("The project will analyze the possibility of using clean energies to\nmeet the plant's needs."),
            create_paragraph("Design/Operation", center_cell_style),
            create_paragraph("Medium", center_cell_style),
            create_paragraph("In Progress", center_cell_style),
            create_paragraph("During Design/\nOperational phase", center_cell_style),
            create_paragraph("Process Engineering", center_cell_style),
            create_paragraph("Electrical Engineering", center_cell_style),
            create_paragraph("Environmental\nEngineering", center_cell_style),
            create_paragraph("Energy policy (OCP)\nProject energy balance"),
            create_paragraph("Energy consumption\nEnergy sanitation\nCarbon footprint"),
            create_paragraph("Ministry of Energy Transition and\nSustainable Development"),
            '', '', 'X', 'X', '', '', '', '', '', '', '', 'X', 'X', '', '', ''
        ],
    ]

    table_data = [header1, header2] + data_rows

      # --- Column Widths ---
    col_widths_proportions = [
    0.06,   # Phase
    0.08,   # Activity/Aspect
    0.06,   # Impacts
    0.06,   # Jurisdiction
    0.06,   # Type
    0.15,   # Legal Requirement
    0.07,   # Date
    0.145,  # Description
    0.12,   # Task
    0.09,   # Responsibility
    0.155   # Comments
]

    # Corrected number of columns to 30
    col_widths = [p * table_width for p in col_widths_proportions]

    # --- Table Styling ---
    style = TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        # Header Row 1 (Blue)
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#002060')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 6),
        # Header Rows 2 & 3 (Red)
        ('SPAN', (11, 0), (29, 0)),
        ('BACKGROUND', (0, 1), (-1, 2), colors.HexColor('#C00000')),
        ('TEXTCOLOR', (0, 1), (-1, 2), colors.white),
        ('FONTNAME', (0, 1), (-1, 2), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 1), (-1, 2), 6),

    ])

    table = Table(table_data, colWidths=col_widths)
    table.setStyle(style)
    return [table]

In [105]:
import os
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Flowable
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import landscape, A3
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from reportlab.pdfgen.canvas import Canvas
from reportlab.pdfbase.pdfmetrics import stringWidth

# This class is unchanged and not used in the final table, but kept for reference.
class VerticalText(Flowable):
    """A custom flowable to draw text rotated by 90 degrees."""
    def __init__(self, text, font_name='Helvetica-Bold', font_size=4):
        Flowable.__init__(self)
        self.text = text
        self.font_name = font_name
        self.font_size = font_size

    def draw(self):
        canvas = self.canv
        canvas.saveState()
        canvas.setFont(self.font_name, self.font_size)
        canvas.rotate(90)
        canvas.drawString(5, -self.font_size, self.text)
        canvas.restoreState()

    def wrap(self, available_width, available_height):
        text_width = stringWidth(self.text, self.font_name, self.font_size)
        return (self.font_size, text_width + 10)

def generate_legal_register_pdf():
    """
    Generates a PDF file with a detailed table matching the provided image.
    """
    output_filename = "legal_register_page3.pdf"
    pagesize = landscape(A3)
    margin = 1.25 * inch
    doc_width, doc_height = pagesize
    table_width = doc_width - 2 * margin

    doc = SimpleDocTemplate(
        output_filename,
        pagesize=pagesize,
        rightMargin=1.2 * inch,
        leftMargin = 1.2* inch,
        topMargin=2.25 * inch,
        bottomMargin=0.75 * inch,
    )


    # --- Paragraph Styles for Cell Content ---
    styles = getSampleStyleSheet()

    # Style for header text (centered)
    header_style = ParagraphStyle(
        name='HeaderStyle',
        parent=styles['Normal'],
        fontName='Helvetica-Bold',
        fontSize=4,
        leading=6,
        alignment=TA_CENTER,
        textColor=colors.white,
    )

    # Style for general cell text (left-aligned)
    cell_style = ParagraphStyle(
    name='CellStyle',
    fontName='Helvetica',
    parent=styles['Normal'],
    alignment=TA_LEFT,
    fontSize=4,
    leading=6,
)

    # Bold cell style
    bold_cell_style = ParagraphStyle(
        name='BoldCellStyle',
        fontName='Helvetica-Bold',
        parent=styles['Normal'],
        alignment=TA_LEFT,
        fontSize=4,
        leading=6,
    )

    # --- Helper functions to create formatted paragraphs ---
    def create_header_paragraph(text, style=header_style):
        return Paragraph(text.replace('\n', '<br/>'), style)

    def create_paragraph(text, style=cell_style):
        return Paragraph(text.replace('\n', '<br/>'), style)

    # --- Table Headers (Two Levels) ---
    header1_str = ['Phase', 'Activity/Aspect', 'Impacts', 'Jurisdiction', 'Type', 'Legal Requirement', 'Date', 'Description', 'Task', 'Responsibility', 'Comments']
    header1 = [create_header_paragraph(h) for h in header1_str]

    # The second header row, with one item meant to span all columns
    header2_text = "General Environmental & Sustainability Regulation"
    header2 = [create_header_paragraph(header2_text)] + [''] * (len(header1_str) - 1)


    # --- All Data Rows Transcribed from the Image ---
    data_rows = [
        [
            create_paragraph("All project phases" , bold_cell_style),
            create_paragraph("Project as a whole"),
            create_paragraph("Environment"),
            create_paragraph("National"),
            create_paragraph("Regulation"),
            create_paragraph('Law No. 11-03 "on the protection and development of the environment"'),
            create_paragraph("Dahir (Law) 1-03-59\n12 May 2003"),
            create_paragraph("Establishes the basic rules and general principles of national policy in the field of environmental protection and development."),
            create_paragraph(""),
            create_paragraph(""),
            create_paragraph("The law sets ESIAs obligations to the project. In fact, this law sets the general framework of environmental protection and applies with various texts, including: liquid discharges, waste, fauna, and flora, of the project at all phases."),
        ],
        [
            create_paragraph("All project phases" ,bold_cell_style),
            create_paragraph("Project as a whole" ),
            create_paragraph("Environment" , ),
            create_paragraph("National"),
            create_paragraph("Regulation"),
            create_paragraph('Framework Law No. 99-12 "on the National Charter for the Environment and Sustainable Development"'),
            create_paragraph("Dahir (Law) 1-14-09\n20 March 2014"),
            create_paragraph("Defines the principles to a sustainable development strategy."),
            create_paragraph(""),
            create_paragraph(""),
            create_paragraph(""),
        ],
        [
            create_paragraph("Pre-construction" , bold_cell_style),
            create_paragraph("Regulatory &\nPermitting"),
            create_paragraph("Regulatory &\nPermitting"),
            create_paragraph("National"),
            create_paragraph("Regulation"),
            create_paragraph('Law No. 12-03 "on environmental impact studies"'),
            create_paragraph("Dahir (Law) 1-03-60\n12 May 2003"),
            create_paragraph("Provides that, the project is subject to the procedure of implementation and conditions of impact studies."),
            create_paragraph("Follow the list of projects subject to EIA and determine if chemical additives plant project applies now"),
            create_paragraph("Environmental Engineer"),
            create_paragraph("This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."),
        ],
        [
            create_paragraph("Pre-construction" , bold_cell_style),
            create_paragraph("Regulatory &\nPermitting"),
            create_paragraph(""),
            create_paragraph("National"),
            create_paragraph("Regulation"),
            create_paragraph('Decree No. 2-04-563 "on the responsibility and functions of the national and regional environmental impact studies committee"'),
            create_paragraph("UG (Decree) n 2-04-563\n04 November 2008"),
            create_paragraph("Defines attributions and functioning of the national and regional committees of environmental impact studies."),
            create_paragraph(""),
            create_paragraph(""),
            create_paragraph(""),
        ]
    ]

    header3_text = "Solid Waste & auxilary products"
    header3 = [create_header_paragraph(header3_text)] + [''] * (len(header1_str) - 1)
    # Assemble the final table data with both headers
    table_data = [header1, header2] + data_rows + [header3]

    # --- Column Widths Adjusted for New Layout ---
    col_widths_proportions = [
        0.07,  # Phase
        0.07,  # Activity/Aspect
        0.07,  # Impacts
        0.05,  # Jurisdiction
        0.05,  # Type
        0.14,  # Legal Requirement
        0.07,  # Date
        0.15,  # Description
        0.12,  # Task
        0.07,  # Responsibility
        0.14,  # Comments
    ]
    col_widths = [p * table_width for p in col_widths_proportions]

    # --- Table Styling ---
    style = TableStyle([
        # General alignment for all cells
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('FONTSIZE', (0, 2), (-1, -1), 7), # Font size for data rows
        ('TOP_PADDING', (0, 0), (-1, -1), 4),
        ('BOTTOM_PADDING', (0, 0), (-1, -1), 4),
        ('LEFT_PADDING', (0, 0), (-1, -1), 4),
        ('RIGHT_PADDING', (0, 0), (-1, -1), 4),

        # Header Row 1 (Blue)
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#002060')),

        # Header Row 2 (Red)
        ('SPAN', (0, 1), (-1, 1)),
        ('BACKGROUND', (0, 1), (-1, 1), colors.HexColor('#C00000')),
    ])

    table = Table(table_data, colWidths=col_widths)
    table.setStyle(style)

    # Build the PDF
    doc.build([table])
    print(f"PDF '{output_filename}' generated successfully.")
    return [table]


# --- Run the generator ---
if __name__ == "__main__":
    generate_commitment_register_pdf()

PDF 'legal_register_page3.pdf' generated successfully.


In [118]:
import os
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import landscape, A3
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from reportlab.pdfgen.canvas import Canvas
from reportlab.pdfbase.pdfmetrics import stringWidth

def generate_legal_register_pdf():
    """
    Generates a PDF file with a detailed table where category headers
    are created dynamically from a structured data source. This approach allows
    for easy addition of new categories and rows, including multiple
    red headers, without repeating code.
    """
    # --- Document Setup ---
    output_filename = "legal_register_dynamically_generated.pdf"
    pagesize = landscape(A3)
    margin = 1.25 * inch
    doc_width, doc_height = pagesize
    table_width = doc_width - 2 * margin

    doc = SimpleDocTemplate(
        output_filename,
        pagesize=pagesize,
        rightMargin=margin,
        leftMargin=margin,
        topMargin=1.5 * inch, # Adjusted margin for better fit
        bottomMargin=margin,
    )

    # --- Paragraph Styles for Cell Content ---
    styles = getSampleStyleSheet()

    # Style for the main blue header row
    header_style = ParagraphStyle(
        name='HeaderStyle',
        parent=styles['Normal'],
        fontName='Helvetica-Bold',
        fontSize=5,  # Slightly larger for readability
        leading=8,
        alignment=TA_CENTER,
        textColor=colors.white
    )

    # Style for the red category headers
    category_header_style = ParagraphStyle(
        name='CategoryHeaderStyle',
        parent=header_style, # Inherits from header_style
        fontSize=6, # Make category headers stand out
        leading=10,
    )

    # Style for standard data cells
    cell_style = ParagraphStyle(
        name='CellStyle',
        parent=styles['Normal'],
        fontName='Helvetica',
        alignment=TA_LEFT,
        fontSize=5,
        leading=7
    )

    # Style for the first column in data rows (bold)
    bold_cell_style = ParagraphStyle(
        name='BoldCellStyle',
        parent=cell_style,
        fontName='Helvetica-Bold',
    )

    # --- Helper functions to create formatted paragraphs ---
    def create_header_paragraph(text, style=header_style):
        return Paragraph(text.replace('\n', '<br/>'), style)

    def create_category_paragraph(text):
        return Paragraph(text.replace('\n', '<br/>'), category_header_style)

    def create_paragraph(text, style=cell_style):
        return Paragraph(text.replace('\n', '<br/>'), style)

    # --- DATA DEFINITION ---
    # This is the core data structure. To add more data or categories,
    # simply add new dictionaries to this list or new rows to an existing category.
    # This structure can be easily loaded from a JSON file.
    structured_data = [
        {
            "category_title": "General Environmental & Sustainability Regulation",
            "rows": [
                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Law No. 11-03 "on the protection and development of the environment"',
                    "Dahir (Law) 1-03-59\n12 May 2003",
                    "Establishes the basic rules and general principles of national policy in the field of environmental protection and development.",
                    "", "",
                    "The law sets ESIAs obligations to the project. In fact, this law sets the general framework of environmental protection and applies with various texts, including: liquid discharges, waste, fauna, and flora, of the project at all phases."
                ],
                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Framework Law No. 99-12 "on the National Charter for the Environment and Sustainable Development"',
                    "Dahir (Law) 1-14-09\n20 March 2014", "Defines the principles to a sustainable development strategy.",
                    "", "", ""
                ],
                [

                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Law No. 11-03 "on the protection and development of the environment"',
                    "Dahir (Law) 1-03-59\n12 May 2003",
                    "Establishes the basic rules and general principles of national policy in the field of environmental protection and development.",
                    "", "",
                    "The law sets ESIAs obligations to the project. In fact, this law sets the general framework of environmental protection and applies with various texts, including: liquid discharges, waste, fauna, and flora, of the project at all phases."
                ],
                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Framework Law No. 99-12 "on the National Charter for the Environment and Sustainable Development"',
                    "Dahir (Law) 1-14-09\n20 March 2014", "Defines the principles to a sustainable development strategy.",
                    "", "", ""
                ],
                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Law No. 11-03 "on the protection and development of the environment"',
                    "Dahir (Law) 1-03-59\n12 May 2003",
                    "Establishes the basic rules and general principles of national policy in the field of environmental protection and development.",
                    "", "",
                    "The law sets ESIAs obligations to the project. In fact, this law sets the general framework of environmental protection and applies with various texts, including: liquid discharges, waste, fauna, and flora, of the project at all phases."
                ],
                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Framework Law No. 99-12 "on the National Charter for the Environment and Sustainable Development"',
                    "Dahir (Law) 1-14-09\n20 March 2014", "Defines the principles to a sustainable development strategy.",
                    "", "", ""
                ],

                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Law No. 11-03 "on the protection and development of the environment"',
                    "Dahir (Law) 1-03-59\n12 May 2003",
                    "Establishes the basic rules and general principles of national policy in the field of environmental protection and development.",
                    "", "",
                    "The law sets ESIAs obligations to the project. In fact, this law sets the general framework of environmental protection and applies with various texts, including: liquid discharges, waste, fauna, and flora, of the project at all phases."
                ],
                [
                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Framework Law No. 99-12 "on the National Charter for the Environment and Sustainable Development"',
                    "Dahir (Law) 1-14-09\n20 March 2014", "Defines the principles to a sustainable development strategy.",
                    "", "", ""
                ],
                [

                    "All project phases", "Project as a whole", "Environment", "National", "Regulation",
                    'Law No. 11-03 "on the protection and development of the environment"',
                    "Dahir (Law) 1-03-59\n12 May 2003",
                    "Establishes the basic rules and general principles of national policy in the field of environmental protection and development.",
                    "", "",
                    "The law sets ESIAs obligations to the project. In fact, this law sets the general framework of environmental protection and applies with various texts, including: liquid discharges, waste, fauna, and flora, of the project at all phases."
                ],

            ],
        },
        {
            "category_title": "Solid Waste & Auxiliary Products",
            "rows": [
                [
                    "Pre-construction", "Regulatory &\nPermitting", "Regulatory &\nPermitting", "National", "Regulation",
                    'Law No. 12-03 "on environmental impact studies"', "Dahir (Law) 1-03-60\n12 May 2003",
                    "Provides that, the project is subject to the procedure of implementation and conditions of impact studies.",
                    "Follow the list of projects subject to EIA and determine if chemical additives plant project applies now",
                    "Environmental Engineer",
                    "This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "", "National", "Regulation",
                    'Decree No. 2-04-563 "on the responsibility and functions of the national and regional environmental impact studies committee"',
                    "UG (Decree) n 2-04-563\n04 November 2008",
                    "Defines attributions and functioning of the national and regional committees of environmental impact studies.",
                    "", "", ""
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "Regulatory &\nPermitting", "National", "Regulation",
                    'Law No. 12-03 "on environmental impact studies"', "Dahir (Law) 1-03-60\n12 May 2003",
                    "Provides that, the project is subject to the procedure of implementation and conditions of impact studies.",
                    "Follow the list of projects subject to EIA and determine if chemical additives plant project applies now",
                    "Environmental Engineer",
                    "This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "", "National", "Regulation",
                    'Decree No. 2-04-563 "on the responsibility and functions of the national and regional environmental impact studies committee"',
                    "UG (Decree) n 2-04-563\n04 November 2008",
                    "Defines attributions and functioning of the national and regional committees of environmental impact studies.",
                    "", "", ""
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "Regulatory &\nPermitting", "National", "Regulation",
                    'Law No. 12-03 "on environmental impact studies"', "Dahir (Law) 1-03-60\n12 May 2003",
                    "Provides that, the project is subject to the procedure of implementation and conditions of impact studies.",
                    "Follow the list of projects subject to EIA and determine if chemical additives plant project applies now",
                    "Environmental Engineer",
                    "This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "", "National", "Regulation",
                    'Decree No. 2-04-563 "on the responsibility and functions of the national and regional environmental impact studies committee"',
                    "UG (Decree) n 2-04-563\n04 November 2008",
                    "Defines attributions and functioning of the national and regional committees of environmental impact studies.",
                    "", "", ""
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "Regulatory &\nPermitting", "National", "Regulation",
                    'Law No. 12-03 "on environmental impact studies"', "Dahir (Law) 1-03-60\n12 May 2003",
                    "Provides that, the project is subject to the procedure of implementation and conditions of impact studies.",
                    "Follow the list of projects subject to EIA and determine if chemical additives plant project applies now",
                    "Environmental Engineer",
                    "This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "", "National", "Regulation",
                    'Decree No. 2-04-563 "on the responsibility and functions of the national and regional environmental impact studies committee"',
                    "UG (Decree) n 2-04-563\n04 November 2008",
                    "Defines attributions and functioning of the national and regional committees of environmental impact studies.",
                    "", "", ""
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "Regulatory &\nPermitting", "National", "Regulation",
                    'Law No. 12-03 "on environmental impact studies"', "Dahir (Law) 1-03-60\n12 May 2003",
                    "Provides that, the project is subject to the procedure of implementation and conditions of impact studies.",
                    "Follow the list of projects subject to EIA and determine if chemical additives plant project applies now",
                    "Environmental Engineer",
                    "This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "", "National", "Regulation",
                    'Decree No. 2-04-563 "on the responsibility and functions of the national and regional environmental impact studies committee"',
                    "UG (Decree) n 2-04-563\n04 November 2008",
                    "Defines attributions and functioning of the national and regional committees of environmental impact studies.",
                    "", "", ""
                ],
                [
                    "Pre-construction", "Regulatory &\nPermitting", "Regulatory &\nPermitting", "National", "Regulation",
                    'Law No. 12-03 "on environmental impact studies"', "Dahir (Law) 1-03-60\n12 May 2003",
                    "Provides that, the project is subject to the procedure of implementation and conditions of impact studies.",
                    "Follow the list of projects subject to EIA and determine if chemical additives plant project applies now",
                    "Environmental Engineer",
                    "This chemical additives plant project is classified under the chemical industry section.\nTherefore, the project is a subject to an EIA in order to obtain the environmental acceptability and the construction permission, which will be conducted by JESA."
                ],

            ],
        },
        # --- ADD YOUR OTHER 6+ CATEGORIES HERE ---
        {
            "category_title": "Water Management & Discharges",
            "rows": [
                 ["Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data"],
            ]
        },
        {
            "category_title": "Air Emissions & Quality",
            "rows": [
                 ["Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data", "Example Data"],
            ]
        },
    ]

    # --- Dynamic Table Assembly ---
    # Define the main (blue) header
    header1_str = ['Phase', 'Activity/Aspect', 'Impacts', 'Jurisdiction', 'Type', 'Legal Requirement', 'Date', 'Description', 'Task', 'Responsibility', 'Comments']
    table_data = [[create_header_paragraph(h) for h in header1_str]]

    # This list will store the row indices of the red headers for styling later
    red_header_indices = []

    # Loop through the structured data to build the table row by row
    for category in structured_data:
        # Add the red category header and record its index in the table
        red_header_indices.append(len(table_data))
        red_header_row = [create_category_paragraph(category['category_title'])] + [''] * (len(header1_str) - 1)
        table_data.append(red_header_row)

        # Add all the data rows for this category
        for row_data in category['rows']:
            # Create Paragraph objects for each cell, applying the bold style to the first cell
            processed_row = [
                create_paragraph(row_data[0], bold_cell_style),
                *[create_paragraph(cell) for cell in row_data[1:]]
            ]
            table_data.append(processed_row)

    # --- Column Widths ---
    col_widths_proportions = [0.07, 0.07, 0.07, 0.05, 0.05, 0.14, 0.07, 0.15, 0.12, 0.07, 0.14]
    col_widths = [p * table_width for p in col_widths_proportions]

    # --- Dynamic Table Styling ---
    style_commands = [
        # General styles for alignment, padding, and grid
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('TOP_PADDING', (0, 0), (-1, -1), 4),
        ('BOTTOM_PADDING', (0, 0), (-1, -1), 4),
        ('LEFT_PADDING', (0, 0), (-1, -1), 4),
        ('RIGHT_PADDING', (0, 0), (-1, -1), 4),

        # Style for the main blue header row
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#002060')),
    ]

    # Dynamically add style commands for each red header row based on the indices we saved
    for row_idx in red_header_indices:
        style_commands.append(('SPAN', (0, row_idx), (-1, row_idx)))
        style_commands.append(('BACKGROUND', (0, row_idx), (-1, row_idx), colors.HexColor('#C00000')))

    # --- Create and Build the PDF ---
    table = Table(table_data, colWidths=col_widths)
    table.setStyle(TableStyle(style_commands))

    # Build the PDF document
    doc.build([table])
    print(f"PDF '{output_filename}' generated successfully.")
    print(f"File saved at: {os.path.abspath(output_filename)}")
    return [table]

# --- Run the generator ---
if __name__ == "__main__":
    generate_legal_register_pdf()


PDF 'legal_register_dynamically_generated.pdf' generated successfully.
File saved at: /content/legal_register_dynamically_generated.pdf


# combine all together

In [119]:

def generate_combined_pdf():
    output_filename = "legal_register_dynamically_generated.pdf"
    pagesize = landscape(A3)
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=pagesize,
        rightMargin=1.2 * inch,
        leftMargin=1.2 * inch,
        topMargin=2.25 * inch,
        bottomMargin=0.75 * inch,
    )


    styles = getSampleStyleSheet()
    doc_width, doc_height = pagesize
    usable_width = doc_width - doc.leftMargin - doc.rightMargin

    # Get both parts
    description_story = generate_commitment_register_landscape()
    table_story = generate_legal_register_pdf()

    # Combine and build
    doc.build(description_story +[PageBreak()]+ table_story, onFirstPage=header_footer, onLaterPages=header_footer)
    print(f"PDF successfully created: {output_filename}")
generate_combined_pdf()

PDF 'legal_register_dynamically_generated.pdf' generated successfully.
File saved at: /content/legal_register_dynamically_generated.pdf
PDF successfully created: legal_register_dynamically_generated.pdf


In [120]:
def build_full_pdf():
    # 1. Generate A4 page
    generate_legal_register_first_page()  # create legal_register_final.pdf

    # 2. Generate landscape multi-page content
    generate_combined_pdf()

    # 3. Merge both into one
    from PyPDF2 import PdfMerger
    merger = PdfMerger()
    merger.append("legal_register_cover_page.pdf")
    merger.append("legal_register_dynamically_generated.pdf")
    merger.write("legal_register.pdf")
    merger.close()

    print("✅ legal_register.pdf successfully created")

# Run it
build_full_pdf()


✅ PDF successfully created: legal_register_cover_page.pdf
PDF 'legal_register_dynamically_generated.pdf' generated successfully.
File saved at: /content/legal_register_dynamically_generated.pdf
PDF successfully created: legal_register_dynamically_generated.pdf
✅ legal_register.pdf successfully created
